# Combining AESO & Weather Data

Okay so now that our two key data sets are proccessed we can integrate them together as one data, which will serve ultimately as our final data set for all our modelling.

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp


spark = (
    SparkSession.builder
    .appName("612 Final Project - Combine AESO & Weather")
    .getOrCreate()
)

weather_spark_bad = (
    spark.read
        .option("header", True)
        .csv("../data/weather_all_areas_hourly.csv")
)

weather_spark_bad.printSchema()
weather_spark_bad.show(5)

root
 |-- area_code: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- temp_c: string (nullable = true)
 |-- precip_mm: string (nullable = true)

+---------+-------------------+------+---------+
|area_code|          timestamp|temp_c|precip_mm|
+---------+-------------------+------+---------+
|        4|2023-11-01 00:00:00|  -4.6|      0.0|
|        4|2023-11-01 01:00:00|  -5.9|      0.0|
|        4|2023-11-01 02:00:00|  -6.5|      0.0|
|        4|2023-11-01 03:00:00|  -6.9|      0.0|
|        4|2023-11-01 04:00:00|  -6.8|      0.0|
+---------+-------------------+------+---------+
only showing top 5 rows


In [7]:
#realized from our schema above that our TIMESTAMP was a string not a TIMESTAMP, so renamed above BAD,
#now we do casting to ensure our schema is perfect
#this will be critical for our joining as we are doing by both TIMESTAMPE and of Course Area_Code

weather_spark = (
    weather_spark
        .withColumn("area_code",  col("area_code").cast("int"))
        .withColumn("timestamp",  to_timestamp("timestamp"))
        .withColumn("temp_c",     col("temp_c").cast("double"))
        .withColumn("precip_mm",  col("precip_mm").cast("double"))
)

weather_spark.printSchema()
weather_spark.show(5)

root
 |-- area_code: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- temp_c: double (nullable = true)
 |-- precip_mm: double (nullable = true)

+---------+-------------------+------+---------+
|area_code|          timestamp|temp_c|precip_mm|
+---------+-------------------+------+---------+
|        4|2023-11-01 00:00:00|  -4.6|      0.0|
|        4|2023-11-01 01:00:00|  -5.9|      0.0|
|        4|2023-11-01 02:00:00|  -6.5|      0.0|
|        4|2023-11-01 03:00:00|  -6.9|      0.0|
|        4|2023-11-01 04:00:00|  -6.8|      0.0|
+---------+-------------------+------+---------+
only showing top 5 rows


In [8]:
aeso_weather = (
    aeso_time_features
        .join(
            weather_spark,
            on=["timestamp", "area_code"],   # common keys
            how="inner"                     # keep rows present in both
        )
)

aeso_weather.printSchema()
aeso_weather.show(10)

NameError: name 'aeso_time_features' is not defined

25/12/03 18:26:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 908293 ms exceeds timeout 120000 ms
25/12/03 18:26:44 WARN SparkContext: Killing executors is not supported by current scheduler.
25/12/03 18:26:46 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at o